<a href="https://colab.research.google.com/github/partho2001/OCR_/blob/main/OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image

# Load IAM dataset (replace 'path_to_iam_dataset' with the actual path)
def load_iam_dataset(dataset_path):
    images = []
    labels = []

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.endswith(".png"):
                img_path = os.path.join(root, file)
                # Assuming the IAM dataset folder structure, where the label is the second-to-last directory
                label = os.path.basename(os.path.dirname(root))
                images.append(img_path)
                labels.append(label)

    return images, labels

dataset_path = '/content/OCR/iam_words'
images, labels = load_iam_dataset(dataset_path)

# Split the dataset into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

def preprocess_image(img_path):
    try:
        img = Image.open(img_path).convert('L')  # Open the image and convert to grayscale
        img = img.resize((64, 64))  # Resize for a better CNN input size
        img_array = img_to_array(img) / 255.0
        return img_array
    except Exception as e:
        print(f"Error processing image {img_path}: {str(e)}")
        return None

# Filter out None entries (failed to load images)
X_train = np.array([preprocess_image(img_path) for img_path in train_images if preprocess_image(img_path) is not None])
y_train = np.array([label for img_path, label in zip(train_images, train_labels) if preprocess_image(img_path) is not None])

print(len(X_train), len(y_train))  # Print debug information

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Split the dataset into training and testing sets for evaluation
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X_train, y_train_encoded, test_size=0.2, random_state=42)

# Create a better CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(np.unique(y_train_encoded)), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_encoded, epochs=10, validation_data=(X_test, y_test_encoded))

# Evaluate the model
y_pred_encoded = np.argmax(model.predict(X_test), axis=1)
accuracy = accuracy_score(y_test_encoded, y_pred_encoded)
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

# Save the trained model
model.save('iam_ocr_model_better_cnn.h5')


Error processing image /content/OCR/iam_words/words/r06/r06-022/r06-022-03-05.png: cannot identify image file '/content/OCR/iam_words/words/r06/r06-022/r06-022-03-05.png'
Error processing image /content/OCR/iam_words/words/r06/r06-022/r06-022-03-05.png: cannot identify image file '/content/OCR/iam_words/words/r06/r06-022/r06-022-03-05.png'
92255 92255
Epoch 1/10
2307/2307 [==============================] - 448s 194ms/step - loss: 3.8134 - accuracy: 0.1165 - val_loss: 3.6128 - val_accuracy: 0.1406
Epoch 2/10
2307/2307 [==============================] - 411s 178ms/step - loss: 3.4695 - accuracy: 0.1626 - val_loss: 3.4111 - val_accuracy: 0.1742
Epoch 3/10
2307/2307 [==============================] - 433s 188ms/step - loss: 3.2354 - accuracy: 0.2092 - val_loss: 3.3217 - val_accuracy: 0.1969
Epoch 4/10
2307/2307 [==============================] - 433s 188ms/step - loss: 3.0220 - accuracy: 0.2526 - val_loss: 3.2551 - val_accuracy: 0.2153
Epoch 5/10
2307/2307 [==============================] 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
